<a href="https://colab.research.google.com/github/Keruthika-N/Milestone-1/blob/main/efficientb0_alzheimer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
import os

# 🔹 Mount Google Drive
drive.mount('/content/drive')

# 🔹 Search for output.zip in MyDrive
zip_path = None
for root, dirs, files in os.walk('/content/drive/MyDrive'):
    if "output.zip" in files:
        zip_path = os.path.join(root, "output.zip")
        break

if zip_path:
    print(f"✅ Found output.zip at: {zip_path}")
else:
    print("❌ output.zip not found. Make sure it's uploaded to Google Drive.")


Mounted at /content/drive
✅ Found output.zip at: /content/drive/MyDrive/output.zip


In [3]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/output.zip"   # path of your dataset
extract_path = "/content/dataset"                # where to extract

# unzip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Dataset unzipped to:", extract_path)


✅ Dataset unzipped to: /content/dataset


In [4]:
import os

train_dir = "/content/dataset"

classes = sorted(os.listdir(train_dir))
print("Classes detected:", classes)


Classes detected: ['Mild Dementia', 'Moderate Dementia', 'Non Demented', 'Very mild Dementia']


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    '/content/dataset',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical'
)

print(train_generator.class_indices)


Found 20000 images belonging to 4 classes.
{'Mild Dementia': 0, 'Moderate Dementia': 1, 'Non Demented': 2, 'Very mild Dementia': 3}


In [6]:
print(os.listdir(extract_path))


['Moderate Dementia', 'Mild Dementia', 'Non Demented', 'Very mild Dementia']


In [7]:
for class_folder in sorted(os.listdir(extract_path)):
    class_path = os.path.join(extract_path, class_folder)
    num_files = len(os.listdir(class_path))
    print(class_folder, "->", num_files, "files")


Mild Dementia -> 5000 files
Moderate Dementia -> 5000 files
Non Demented -> 5000 files
Very mild Dementia -> 5000 files


In [8]:
!pip install split-folders
import splitfolders
input_dir = "/content/dataset"
output_dir = "/content/oasis_split_colab"

# 70% train, 15% val, 15% test
splitfolders.ratio(input_dir, output=output_dir, seed=42, ratio=(0.7, 0.15, 0.15))

print("✅ Splitting done! Folder structure created at:", output_dir)
print("Subfolders:", os.listdir(output_dir))


Copying files: 20000 files [00:02, 7856.01 files/s]

✅ Splitting done! Folder structure created at: /content/oasis_split_colab
Subfolders: ['train', 'val', 'test']


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input

IMG_SIZE = 176
BATCH_SIZE = 32
work_dir = "/content/oasis_split_colab"

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=15,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.9, 1.1],
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_gen = train_datagen.flow_from_directory(
    f"{work_dir}/train",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

val_gen = val_datagen.flow_from_directory(
    f"{work_dir}/val",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

test_gen = test_datagen.flow_from_directory(
    f"{work_dir}/test",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)


Found 14000 images belonging to 4 classes.
Found 3000 images belonging to 4 classes.
Found 3000 images belonging to 4 classes.


In [10]:
import tensorflow as tf
print("Available GPU:", tf.config.list_physical_devices('GPU'))


Available GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [11]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
import tensorflow as tf

IMG_SIZE = 176
NUM_CLASSES = 4

base_model = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

base_model.trainable = False

base_model.summary()


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "efficientnetb0"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 176, 176,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 176, 176,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 176, 176,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 176, 176,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 177, 177,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 88, 88,    │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 88, 88,    │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 88, 88,    │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 88, 88,    │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 88, 88,    │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 88, 88,    │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 88, 88,    │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 88, 88,    │        512 │ block1a_se_excit

 Total params: 4,049,571 (15.45 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 4,049,571 (15.45 MB)

In [12]:
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)

x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.3)(x)

outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = models.Model(inputs=base_model.input, outputs=outputs)

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 176, 176,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 176, 176,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 176, 176,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 176, 176,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 177, 177,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 88, 88,    │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 88, 88,    │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 88, 88,    │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 88, 88,    │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 88, 88,    │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 88, 88,    │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 88, 88,    │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 88, 88,    │        512 │ block1a_se_excit

 Total params: 4,219,175 (16.09 MB)

 Trainable params: 167,044 (652.52 KB)

 Non-trainable params: 4,052,131 (15.46 MB)

In [13]:
from tensorflow.keras import layers, models

NUM_CLASSES = 4   # you already confirmed 4 classes

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)

x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.3)(x)

outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = models.Model(inputs=base_model.input, outputs=outputs)

model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 176, 176,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 176, 176,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 176, 176,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 176, 176,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 177, 177,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 88, 88,    │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 88, 88,    │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 88, 88,    │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 88, 88,    │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 88, 88,    │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 88, 88,    │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 88, 88,    │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 88, 88,    │        512 │ block1a_se_excit

 Total params: 4,219,175 (16.09 MB)

 Trainable params: 167,044 (652.52 KB)

 Non-trainable params: 4,052,131 (15.46 MB)

In [14]:
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [15]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 201s 390ms/step - accuracy: 0.5827 - loss: 1.0410 - val_accuracy: 0.7580 - val_loss: 0.6114
Epoch 2/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 135s 309ms/step - accuracy: 0.7504 - loss: 0.6141 - val_accuracy: 0.8120 - val_loss: 0.4744
Epoch 3/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 134s 306ms/step - accuracy: 0.7606 - loss: 0.5721 - val_accuracy: 0.8160 - val_loss: 0.4330
Epoch 4/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 135s 309ms/step - accuracy: 0.7885 - loss: 0.5215 - val_accuracy: 0.8180 - val_loss: 0.4431
Epoch 5/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 134s 306ms/step - accuracy: 0.7995 - loss: 0.4898 - val_accuracy: 0.8463 - val_loss: 0.3678


In [16]:
from google.colab import drive
drive.mount('/content/drive')

model.save("/content/drive/MyDrive/efficientnet_frozen.h5")
print("✅ Frozen EfficientNet saved successfully!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Frozen EfficientNet saved successfully!


In [17]:
base_model.trainable = True

for layer in base_model.layers[:-30]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [18]:
history_fine = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=3
)


Epoch 1/3
438/438 ━━━━━━━━━━━━━━━━━━━━ 190s 368ms/step - accuracy: 0.6425 - loss: 0.8999 - val_accuracy: 0.7570 - val_loss: 0.6122
Epoch 2/3
438/438 ━━━━━━━━━━━━━━━━━━━━ 136s 311ms/step - accuracy: 0.7066 - loss: 0.7095 - val_accuracy: 0.7823 - val_loss: 0.5492
Epoch 3/3
438/438 ━━━━━━━━━━━━━━━━━━━━ 136s 310ms/step - accuracy: 0.7275 - loss: 0.6555 - val_accuracy: 0.7987 - val_loss: 0.4980


In [19]:
# Unfreeze the last 15 layers
base_model.trainable = True
for layer in base_model.layers[:-15]:
    layer.trainable = False

# Compile with a very small learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-6),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Optional callbacks
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)
mc = ModelCheckpoint('/content/drive/MyDrive/tree_model_finetuned.h5',
                     monitor='val_accuracy', save_best_only=True)

# Train for 5–7 epochs
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=7,
    callbacks=[es, mc]
)


Epoch 1/7
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step - accuracy: 0.7482 - loss: 0.5941

438/438 ━━━━━━━━━━━━━━━━━━━━ 183s 362ms/step - accuracy: 0.7482 - loss: 0.5941 - val_accuracy: 0.8020 - val_loss: 0.4914
Epoch 2/7
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step - accuracy: 0.7495 - loss: 0.5999

438/438 ━━━━━━━━━━━━━━━━━━━━ 135s 309ms/step - accuracy: 0.7495 - loss: 0.5999 - val_accuracy: 0.8050 - val_loss: 0.4853
Epoch 3/7
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step - accuracy: 0.7446 - loss: 0.6123

438/438 ━━━━━━━━━━━━━━━━━━━━ 135s 309ms/step - accuracy: 0.7447 - loss: 0.6123 - val_accuracy: 0.8070 - val_loss: 0.4842
Epoch 4/7
438/438 ━━━━━━━━━━━━━━━━━━━━ 133s 303ms/step - accuracy: 0.7539 - loss: 0.5893 - val_accuracy: 0.8037 - val_loss: 0.4844
Epoch 5/7
438/438 ━━━━━━━━━━━━━━━━━━━━ 136s 309ms/step - accuracy: 0.7522 - loss: 0.5899 - val_accuracy: 0.8063 - val_loss: 0.4814
Epoch 6/7
438/438 ━━━━━━━━━━━━━━━━━━━━ 135s 309ms/step - accuracy: 0.7502 - loss: 0.6030 - val_accuracy: 0.8067 - val_loss: 0.4786


In [27]:
from tensorflow.keras.utils import img_to_array, load_img
import numpy as np
import os
import tensorflow as tf

def predict_folder_summary(folder_path, model, class_indices, img_size=176):
    if not os.path.exists(folder_path):
        print(f"❌ Folder not found: {folder_path}")
        return

    inv_class_indices = {v: k for k, v in class_indices.items()}

    image_files = [f for f in os.listdir(folder_path)
                   if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    if len(image_files) == 0:
        print("⚠️ No images found in folder")
        return

    class_counts = {class_name: 0 for class_name in class_indices.keys()}

    for img_name in image_files:
        img_path = os.path.join(folder_path, img_name)

        # Load & preprocess correctly for EfficientNet
        img = load_img(img_path, target_size=(img_size, img_size))
        x = img_to_array(img)
        x = tf.keras.applications.efficientnet.preprocess_input(x)  # ✅ correct preprocessing
        x = np.expand_dims(x, axis=0)

        preds = model.predict(x, verbose=0)[0]
        pred_class_idx = np.argmax(preds)
        pred_class_name = inv_class_indices[pred_class_idx]

        class_counts[pred_class_name] += 1

    # Print summary
    print(f"\n📂 Folder: {os.path.basename(folder_path)}")
    print(f"Total images: {len(image_files)}\n")
    print("Predicted class counts:")
    for cls, count in class_counts.items():
        print(f"{cls:<20} -> {count}")
folder = "/content/drive/MyDrive/oasis_split_colab/test/Mild Dementia"
predict_folder_summary(folder, model, test_gen.class_indices, img_size=176)



📂 Folder: Mild Dementia
Total images: 750

Predicted class counts:
Mild Dementia        -> 695
Moderate Dementia    -> 1
Non Demented         -> 26
Very mild Dementia   -> 28


In [ ]:
# 1️⃣ Mount Google Drive
from tensorflow.keras.utils import img_to_array, load_img
import numpy as np
import os
from google.colab import drive
drive.mount('/content/drive')

# 2️⃣ Load the saved model
from tensorflow.keras.models import load_model

# Path to your saved frozen EfficientNet model
model_path = "/content/drive/MyDrive/efficientnet_frozen.h5"
model = load_model(model_path)
print("✅ Model loaded successfully!")